<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/Beta_and_Volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

선택된 종목의 시장베타와 변동성을 알아본다.<BR>
베타를 계산할 때는 통상 일간 수익률의 변동성을 사용한다.

먼저 finance-datareader를 설치한다.

In [1]:
pip install -q finance-datareader

먼저 필요한 모듈을 임포트 한다. 선형회귀를 위해 stats도 임포트 한다.

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
from scipy import stats

불러올 종목의 리스트를 만들고, 제일 마지막에는 KOSPI지수를 불러온다.

In [3]:
names = ["Samsung",  "SK hynix", "SS Bio", "LG Chem", "SS SDI", "HD Motors", "KIA", "POSCO Holdings","NAVER", "KOSPI"]
tickers = [ "005930",  "000660", "207940", "051910", "006400", "005380", "000270", "005490",  "035420" , "KS11"]

읽어온 종목 주가를 저장할 데이터프레임을 생성한다.

In [4]:
df = pd.DataFrame()

tikcers에 있는 종목을 읽어온다.

In [5]:
for ticker in tickers:
  df[ticker] = fdr.DataReader( ticker, "2022-07-01", "2023-08-31").Close

열을 ticker 대신 종목 이름으로 대체하여 가독성을 높인다.

In [6]:
df.columns = names

각 종목의 베타와 변동성을 저장할 배열을 선언한다.<BR>
베타는 beta에 변동성(표준편차)은 std에 저장한다.

In [7]:
beta = np.zeros( len(names) - 1 )  # 마지막은 코스피 지수므로, 전체 길이에서 1을 차감
std = np.zeros( len(names) - 1 )

먼저 일간 수익률을 계산하고 이를 ret라는 데이터프레임에 저장한다.

In [27]:
ret = df.pct_change().fillna(0)  # 일별 수익률을 구하고 빈곳은 0으로 채움

루프를 돌면서 선형회귀를 통해 베타값을 찾고, 동시에 일 평균 변동성도 계산한다.<BR>
Ri = b + beta*Rm + error (즉 Y = ax+ b에서 각 자산은 Y, 시장은 X)

In [35]:
for i in  range(len(tickers)-1):
  beta[i], intercept, r_value, p_value, std_err = stats.linregress(ret[names[-1]], ret[names[i]])
  std[i] = ret[names[i]].std()

이제 도식화 해 본다.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter( std, beta )

# 범례 표기
for i, name in enumerate(names[:-1]):
    plt.annotate( f"{name}\n({std[i]:.3f}:{beta[i]:.3f})" , (std[i], beta[i]), xytext=(5,-10), textcoords='offset points')

# 그래프 제목과 라벨
plt.title('Beta vs. Volatility')
plt.xlabel('Volatility')
plt.ylabel('Beta')

plt.axhline(y=1, color='lightgray', linestyle='--')  # Beta가 1인 지점에 점선을 그린다.